## Парсинг данных

In [1]:
# импортирую нужные библиотеки

import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import numpy as np
import concurrent.futures
import itertools

In [217]:
# проверяю, что данные со страницы парсятся

url = requests.get('https://m2.ru/moskva/nedvizhimost/kupit-kvartiru/rayon-ramenki/') 
bs_url = BS(url.content, 'html.parser')

In [220]:
# ищу ссылки на все объявления о квартирах

flat_links_all = []
for i in range(202):
    url_i = requests.get('https://m2.ru/moskva/nedvizhimost/kupit-kvartiru/rayon-ramenki/?pageNumber='+str(i))
    bs_url_i = BS(url_i.content, 'html.parser')
    flats_i = bs_url_i.findAll('div', class_ = 'ClOfferSnippet__gallery')
    for flat in flats_i:
        flat_links_all.append(flat.find('a').get('href'))

In [240]:
# сам парсинг нужных признаков

name = []
nearest_subway = []
min_to_subway = []
dist_to_centre = []
price = []
price_for_metr = []
rooms = []
floor = []
all_floor = []
square = []
live_square = []
kitchen_square = []
urls = []

for url in range(len(all_links_for_flats)):

    url = requests.get(all_links_for_flats[url])
    bs_url = BS(url.content, 'html.parser')
    try:        
        name.append(bs_url.find('h1', itemprop = 'name').text.replace('\xa0', "")) 
    except Exception as e:
        name.append('')
    try:        
        nearest_subway.append(bs_url.find('div', class_='OfferRouteCardList').find_all('a', class_='SubwayStation__link')[0].text)
    except Exception as e:
        nearest_subway.append('')
    try:        
        min_to_subway.append(bs_url.find('div', class_='OfferRouteCardList').find_all('div', class_='colors-named-module__secondary___eb0c51 fonts-module__primary___73abfc')[0].text)
    except Exception as e:
        min_to_subway.append('')
    try:        
        dist_to_centre.append(bs_url.find('div', class_ ='ClClickableAddress__links').find_all('span')[-1].text.replace('\xa0', ""))
    except Exception as e:
        dist_to_centre.append('')
    try:        
        price.append(bs_url.find('span', itemprop = 'price').get('content'))
    except Exception as e:
        price.append('')   
    try:        
        price_for_metr.append(bs_url.find('div', class_ = 'OfferCard__price-detail').find_all('div', class_='colors-named-module__secondary___eb0c51 fonts-module__primary-alone___d9c4d7')[0].text.replace('\xa0', ""))
    except Exception as e:
        price_for_metr.append('')
        
        
    try:
        test1 = bs_url.find('div', class_ = 'grid-module__container___ef8782 grid-module__gap___08da08').find_all('a', class_='fonts-module__primary___73abfc DescriptionCell__value')
    except Exception as e:
        rooms.append('')
        floor.append('')
        all_floor.append('') 
        
    try:    
        test2 = bs_url.find('div', class_ = 'grid-module__container___ef8782 grid-module__gap___08da08').find_all('div', class_='fonts-module__primary___73abfc DescriptionCell__value')
    except Exception as e:
        square.append('')
        live_square.append('')
        kitchen_square.append('')
        
    if len(test1) == 2:   
        rooms.append(test1[0].text)
        floor1 = test1[1].text.split(' из ')
        if len(floor1) == 2:
            floor.append(floor1[0])
            all_floor.append(floor1[1])
        else:
            floor.append('')
            all_floor.append('')           
    else:
        rooms.append('')
        floor.append('')
        all_floor.append('')

    if len(test2) == 3:
        square.append(test2[0].text.replace('\xa0', ""))
        live_square.append(test2[1].text.replace('\xa0', ""))
        kitchen_square.append(test2[2].text.replace('\xa0', ""))
    else:
        square.append('')
        live_square.append('')
        kitchen_square.append('')
    

In [324]:
# сохраняю нужные признаки с адекватными названиями в таблицу

df = pd.DataFrame({'Название': name, 'Ближайшее метро': nearest_subway, 'Расстояние до центра, км': dist_to_centre, 'Цена за квадратный метр, руб': price_for_metr, 'Количество комнат': rooms, 'Этаж': floor,
                   'Количество этажей в доме': all_floor, 'Площадь квартиры, м^2': square, 'Жилая площадь, м^2': live_square, 'Площадь кухни, м^2': kitchen_square, 'Цена, руб': price})
df

,Название,Ближайшее метро,"Расстояние до центра, км","Цена за квадратный метр, руб",Количество комнат,Этаж,Количество этажей в доме,"Площадь квартиры, м^2","Жилая площадь, м^2","Площадь кухни, м^2","Цена, руб"
0,"Продажа 3-комнатной квартиры 130м², 11/22этаж",Ломоносовский проспект,"•до центра 8,8 км",423077₽/м²,3-комнатная,11,22,130м²,70м²,20м²,55000000
1,"Продажа 4-комнатной квартиры 106м², 3/10этаж",Аминьевская,"•до центра 11,2 км",646226₽/м²,4-комнатная,3,10,,,,68500000
2,"Продажа многокомнатной квартиры 136,9м², 8/14этаж",Минская,"•до центра 8,8 км",745069₽/м²,5-комнатная,8,14,"136,9м²","72,7м²",12м²,102000000
3,"Продажа 2-комнатной квартиры 75м², 7/10этаж",Аминьевская,"•до центра 11,2 км",440000₽/м²,2-комнатная,7,10,75м²,35м²,21м²,33000000
4,"Продажа 3-комнатной квартиры 118м², 9/18этаж",Раменки,"•до центра 9,7 км",507627₽/м²,3-комнатная,9,18,118м²,75м²,20м²,59900000
...,...,...,...,...,...,...,...,...,...,...,...
3619,"Продажа 4-комнатной квартиры 107м², 11/52этаж",Раменки,"•до центра 10,7 км",925234₽/м²,4-комнатная,11,52,,,,99000000
3620,"Продажа 1-комнатной квартиры 45,9м², 22/49этаж",Мичуринский проспект,"•до центра 11,3 км",545500₽/м²,,,,,,,25038450
3621,"Продажа 3-комнатной квартиры 99,7м², 46/49этаж",Мичуринский проспект,"•до центра 11,3 км",504800₽/м²,,,,,,,50328560
3622,"Продажа многокомнатной квартиры 100м², 6/14этаж",Минская,"•до центра 8,8 км",700000₽/м²,5-комнатная,6,14,100м²,60м²,12м²,70000000


In [341]:
# сохраняю таблицу в файл, чтобы использовать его в дальнейшем исследовании

df.to_csv('data.csv', index = False)